In [1]:
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.agents.sarsa import SARSAAgent
from rl.memory import EpisodeParameterMemory

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy
from rl.policy import EpsGreedyQPolicy
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [2]:
env = gym.make('gym_ent:ent-v1')

print(env.observation_space)
print(env.action_space)

nb_actions = env.action_space.n
obs_dim = env.observation_space.shape
#print(nb_actions)
#print(obs_dim)
#env.observation_space.shape

Box(8, 2)
Discrete(4)


In [3]:
# Option 1 : Simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
_________________________________________________________________
activation (Activation)      (None, 4)                 0         
Total params: 68
Trainable params: 68
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=200, train_interval=10)
#sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=2000, train_interval=100)
sars.compile(optimizer='adam')

In [5]:
sars.fit(env, nb_steps=2000, visualize=False, verbose=1)
#sars.fit(env, nb_steps=400000, visualize=False, verbose=1)
# After training is done, we save the best weights.
#sars.save_weights(f'sars_{ENV_NAME}_params.h5f', overwrite=True)

Training for 2000 steps ...
Interval 1 (0 steps performed)
  177/10000 [..............................] - ETA: 5s - reward: 7.5650  

/home/lkoziol/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 1981/10000 [====>.........................] - ETA: 7s - reward: 6.7860done, took 1.953 seconds


In [13]:
sars.test(env, nb_episodes=2, visualize=True)
# 8/25

Testing for 2 episodes ...


TclError: invalid command name ".!frame.!canvas"

In [7]:
# memory = EpisodeParameterMemory(limit=2000, window_length=1)
# cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
#                batch_size=50, nb_steps_warmup=500, train_interval=50, elite_frac=0.05)
# cem.compile()

# cem.fit(env, nb_steps=10000, visualize=False, verbose=1)

In [8]:
#cem.test(env, nb_episodes=5, visualize=True)

In [10]:
env.render(close=True)

In [12]:
env.reset()

array([[ 8,  7],
       [ 3,  3],
       [ 2, 12],
       [13,  3],
       [13, 11],
       [ 6,  7],
       [ 8,  5],
       [ 8, 10]])